## **Sequence-to-sequence learning : FR to EN translation** - revision 02

### **1. Small_vocab translation**

In [1]:
import os
import time

def load_data(path):

    # Nombre maximum de lignes à renvoyer
    max_lines = 140000
    
    input_file = os.path.join(path)
    with open(input_file, "r",  encoding="utf-8") as f:
        data = f.read()
        
    # On convertit les majuscules en minulcule
    data = data.lower()
    
    data = data.split('\n')
    return data[:min(len(data),max_lines)]

# Indication de calculer le score Bleu
bleu_score_to_do = True

# Chargement des textes dans les 2 langues (max lignes = max_lines)
src = 'fr'
tgt = 'en'
txt_tgt = load_data('../data/small_vocab_'+tgt)
txt_src = load_data('../data/small_vocab_'+src)

text_pairs = []
text_tgt_to_vec = []
text_pairs_to_vec = []
for line in range(len(txt_src)):
    text_pairs.append((txt_src[line], "[start] " + txt_tgt[line] + " [end]" ))
    text_tgt_to_vec.append(txt_tgt[line])
    

In [2]:
import random
random.Random(4).shuffle(text_pairs)
random.Random(4).shuffle(text_tgt_to_vec)
print(text_pairs[0])
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
train_tgt_to_vec = text_tgt_to_vec[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

('californie est généralement occupé en décembre , mais il est sec en mars .', '[start] california is usually busy during december , but it is dry in march . [end]')


**Vectorizing the English and French text pairs**

In [3]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    lowercase=tf.strings.regex_replace(lowercase, "[à]", "a")
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

def save_vocab(file_path, vocab):
    with open(file_path, "w",  encoding="utf-8" ) as file:
        for word in vocab:
            file.write(word+'\n')
    return

def load_vocab(file_path):
    with open(file_path, "r",  encoding="utf-8") as file:
        return file.read().split('\n')[:-1]


vocab_size = 15000
sequence_length = 30

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    # Si les fichier vocab existe déjà, enlever la ligne de commentaire ci-dessous
    vocabulary = load_vocab("../data/vocab_"+src+".txt"),
    standardize=custom_standardization,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    # Si les fichier vocab existe déjà, enlever la ligne de commentaire ci-dessous
    vocabulary = load_vocab("../data/vocab_"+tgt+".txt"),
    standardize=custom_standardization,
)
train_src_texts = [pair[0] for pair in train_pairs]
train_tgt_texts = [pair[1] for pair in train_pairs]

In [4]:
# Si les fichier vocab existe déjà, il faut executer le code ci-dessous.
# Sinon il ne faut pas l'executer 

#buff = train_src_texts[0]
#train_tgt_to_vec[0] = "[start] [end] "+train_tgt_to_vec[0]
#train_src_texts[0] = "[start] [end] "+ train_src_texts[0]
#source_vectorization.adapt(train_src_texts)
#target_vectorization.adapt(train_tgt_to_vec)
#train_src_texts[0] = buff
#save_vocab("../data/vocab_"+tgt+".txt" , target_vectorization.get_vocabulary()[2:])
#save_vocab("../data/vocab_"+src+".txt" , source_vectorization.get_vocabulary()[2:])

**Preparing datasets for the translation task**

In [5]:
batch_size = 64

def format_dataset(source, target):
    source = source_vectorization(source)
    target = target_vectorization(target)
    return ({
        src: source,
        tgt: target[:, :-1],
    }, target[:, 1:])

def make_dataset(pairs):
    src_texts, tgt_texts = zip(*pairs)
    tgt_texts = list(tgt_texts)
    src_texts = list(src_texts)
    dataset = tf.data.Dataset.from_tensor_slices((src_texts, tgt_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [6]:
for inputs, targets in train_ds.take(1):
    print(f"inputs[tgt].shape: {inputs[tgt].shape}")
    print(f"inputs[src].shape: {inputs[src].shape}")
    print(f"targets.shape: {targets.shape}")

inputs[tgt].shape: (64, 30)
inputs[src].shape: (64, 30)
targets.shape: (64, 30)


### **2. Sequence-to-sequence learning with RNNs**

**GRU-based encoder**

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name=src)
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [8]:
past_target = keras.Input(shape=(None,), dtype="int64", name=tgt)
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [9]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[stop_early])

Epoch 1/15
1508/1508 [==============================] - 129s 76ms/step - loss: 0.2397 - accuracy: 0.8338 - val_loss: 0.0227 - val_accuracy: 0.9747
Epoch 2/15
1508/1508 [==============================] - 111s 74ms/step - loss: 0.0222 - accuracy: 0.9761 - val_loss: 0.0171 - val_accuracy: 0.9787
Epoch 3/15
1508/1508 [==============================] - 111s 74ms/step - loss: 0.0183 - accuracy: 0.9786 - val_loss: 0.0155 - val_accuracy: 0.9796
Epoch 4/15
1508/1508 [==============================] - 111s 73ms/step - loss: 0.0169 - accuracy: 0.9797 - val_loss: 0.0151 - val_accuracy: 0.9803
Epoch 5/15
1508/1508 [==============================] - 111s 74ms/step - loss: 0.0161 - accuracy: 0.9803 - val_loss: 0.0146 - val_accuracy: 0.9808
Epoch 6/15
1508/1508 [==============================] - 2809s 2s/step - loss: 0.0156 - accuracy: 0.9808 - val_loss: 0.0150 - val_accuracy: 0.9807
Epoch 7/15
1508/1508 [==============================] - 107s 71ms/step - loss: 0.0153 - accuracy: 0.9809 - val_loss: 0.

**Saving of the trained RNN model**

In [10]:
# Attention les instructions suivantes (en commentaire) génèrent un fichier de taille > 100 Mo, ce qui bloque la synchronisation avec Github (commit)
seq2seq_rnn.save("../data/seq2seq_rnn-model-"+src+"-"+tgt+".h5")
#seq2seq_rnn.save("../data/seq2seq_rnn-model_no_opt-"+src+"-"+tgt+".h5", include_optimizer=False)
#seq2seq_rnn.save_weights("../data/seq2seq_rnn-model-"+src+"-"+tgt+".weights.h5")

# Voici une instruction qui permet de passer sous la barre des 100 Mo.....
from filesplit.split import Split
Split("../data/seq2seq_rnn-model-"+src+"-"+tgt+".h5", "../data/rnn_"+src+"-"+tgt+"_split").bysize(66846720)

# seq2seq_rnn = keras.models.load_model("../data/seq2seq_rnn-model-"+src+"-"+tgt+".h5")

**Translating new sentences with our RNN encoder and decoder**

In [27]:
import numpy as np
tgt_vocab = target_vectorization.get_vocabulary()
tgt_index_lookup = dict(zip(range(len(tgt_vocab)), tgt_vocab))
max_decoded_sentence_length = 30

def decode_sequence_rnn(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence], verbose=0)
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = tgt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_src_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_src_texts)
    print("-")
    print(src.upper()+"  ",input_sentence)
    print(tgt.upper()+"->",decode_sequence_rnn(input_sentence))

-
FR   l' inde est jamais froid en juin , mais il est parfois merveilleux en avril .
EN-> india is never chilly during june but it is sometimes wonderful in april
-
FR   les états-unis est jamais à sec au mois d' août , mais il est parfois calme en mars .
EN-> the united states is never dry during august but it is sometimes quiet in march
-
FR   notre fruit préféré moins est la banane , mais votre moins préférée est la chaux .
EN-> our least favorite fruit is the banana but your least favorite is the lime
-
FR   paris est parfois chaud en décembre , mais il est jamais agréable au printemps .
EN-> paris is sometimes warm during december but it is never nice in spring
-
FR   quand est -elle allée sur le terrain de football ?
EN-> when did she go to the football field
-
FR   aiment-ils la californie ?
EN-> do they like california is usually
-
FR   l' inde est jamais chaud au mois de mars , mais il est doux en été .
EN-> india is never warm during march but it is mild in summer
-
FR   elle

In [37]:
if src=='fr':
    your_sentence1 = "paris est pluvieux pendant l'été, mais la france n'est jamais magnifique en hiver" 
    your_sentence2 = "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ." 
else:
    your_sentence1 = "paris is rainy during summer , and france is never wonderful in winter"
    your_sentence2 = "new jersey is sometimes quiet during autumn , and it is snowy in april ."
    
print(tgt.upper()+"->",decode_sequence_rnn(your_sentence1))
print(tgt.upper()+"->",decode_sequence_rnn(your_sentence2))

EN-> paris is rainy during winter but it is never wonderful in winter
EN-> new jersey is sometimes quiet during autumn and it is snowy in april


### **3. Sequence-to-sequence learning with Transformer**

#### The Transformer decoder

**The `TransformerDecoder`**

In [17]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [18]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [19]:
from keras_nlp.layers import TransformerEncoder
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name=src)
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingSource')(encoder_inputs)
encoder_outputs = TransformerEncoder(dense_dim, num_heads, name='encoder_outputs')(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name=tgt)
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingTarget')(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads,  name='transformerDecoder')(x, encoder_outputs)
x = layers.Dropout(0.5, name='dropout')(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax", name='decoder_outputs')(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [20]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
transformer.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[stop_early])

Epoch 1/15
1508/1508 [==============================] - 91s 59ms/step - loss: 0.1379 - accuracy: 0.9160 - val_loss: 0.0223 - val_accuracy: 0.9742
Epoch 2/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0260 - accuracy: 0.9733 - val_loss: 0.0273 - val_accuracy: 0.9715
Epoch 3/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0231 - accuracy: 0.9749 - val_loss: 0.0193 - val_accuracy: 0.9765
Epoch 4/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0209 - accuracy: 0.9760 - val_loss: 0.0170 - val_accuracy: 0.9777
Epoch 5/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0194 - accuracy: 0.9767 - val_loss: 0.0181 - val_accuracy: 0.9771
Epoch 6/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0184 - accuracy: 0.9773 - val_loss: 0.0171 - val_accuracy: 0.9778
Epoch 7/15
1508/1508 [==============================] - 91s 60ms/step - loss: 0.0178 - accuracy: 0.9778 - val_loss: 0.0163 -

**Saving of the trained Transformer model**

In [21]:
for i in range(len(transformer.weights)):
    transformer.weights[i]._handle_name = transformer.weights[i].name + "_" + str(i)
    
transformer.save("../data/transformer-model-"+src+"-"+tgt+".h5", include_optimizer=False)
transformer.save_weights("../data/transformer-model-"+src+"-"+tgt+".weights.h5")


**Translating new sentences with our Transformer model**

In [35]:
import numpy as np
tgt_vocab = target_vectorization.get_vocabulary()
tgt_index_lookup = dict(zip(range(len(tgt_vocab)), tgt_vocab))
max_decoded_sentence_length = 30

def decode_sequence_transf(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = tgt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_src_texts = [pair[0] for pair in test_pairs]
test_tgt_texts = [pair[1][8:-6] for pair in test_pairs]

# Removing Punctuations (we keep the . character)
REPLACEMENTS = [(".", ""),
                (",", ""),
                (";", ""),
                (":", ""),
                ("?", ""),
                ('"', ""),
                ("-", " "),
                ("'", " ")
               ]
for i in range(len(test_tgt_texts)):
    for old, new in REPLACEMENTS:
        test_tgt_texts[i] = test_tgt_texts[i].replace(old, new)
        
for _ in range(20):
    n = random.randrange(num_val_samples)
    input_sentence = test_src_texts[n]
    translation = test_tgt_texts[n]
    print("-")
    print(src.upper()+" ",input_sentence)
    print(tgt.upper()+"  ",translation)
    print(tgt.upper()+"->",decode_sequence_transf(input_sentence))

-
FR  california est relaxant pendant l' été , et il ne fait jamais froid en août .
EN   california is relaxing during summer  and it is never cold in august 
EN-> california is relaxing during summer and it is never cold in august
-
FR  les états-unis est généralement pluvieux en septembre , et il est généralement agréable en juin .
EN   the united states is usually rainy during september  and it is usually nice in june 
EN-> the united states is usually rainy during september and it is usually nice in june
-
FR  californie est jamais de neige en octobre , mais il est parfois relaxant en mars .
EN   california is never snowy during october  but it is sometimes relaxing in march 
EN-> california is never snowy during october but it is sometimes relaxing in march
-
FR  elle déteste les citrons , les pommes et les mangues .
EN   she dislikes lemons  apples  and mangoes 
EN-> she dislikes lemons apples and mangoes
-
FR  son animal préféré était moins ce requin .
EN   his least favorite an

<br></br>
**You turn to play:** Enter a sentence..

In [36]:
if src=='fr':
    your_sentence1 = "paris est pluvieux pendant l'été, mais la france n'est jamais magnifique en hiver" 
    your_sentence2 = "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ." 
else:
    your_sentence1 = "paris is rainy during summer , and france is never wonderful in winter"
    your_sentence2 = "new jersey is sometimes quiet during autumn , and it is snowy in april ."
    

print(tgt.upper()+"->",decode_sequence_transf(your_sentence1))
print(tgt.upper()+"->",decode_sequence_transf(your_sentence2))

EN-> paris is rainy during summer but it is never warm in winter
EN-> new jersey is sometimes quiet during autumn and it is snowy in april


**Qualité de traduction seq2seq sur le corpus Test par les 2 méthodes (300 premieres phrases)**

In [25]:
from sacrebleu import sentence_bleu, corpus_bleu

# Taille du corpus d'évaluation
n = 300

def corpus_translation(method):
    translation = []
    test_src_texts = [pair[0] for pair in test_pairs]
    for txt in test_src_texts[:n]:
        if method == 'RNN':
            translation.append(decode_sequence_rnn(txt))
        else:
            translation.append(decode_sequence_transf(txt))
    return translation

if bleu_score_to_do:
    tps1 = time.time()
    tr_RNN = corpus_translation('RNN')
    translation_quality_RNN = corpus_bleu(tr_RNN, [test_tgt_texts[:n]]).score
    print(f'Score Bleu de la traduction du corpus Test par la méthode RNN = {translation_quality_RNN:.1f}%')
    tps2 = time.time()
    print(" -> durée de traduction et de calcul du score :",int(tps2-tps1)," secondes\n")
    tr_Transformer = corpus_translation('Transformer')
    translation_quality_Transformer = corpus_bleu(tr_Transformer, [test_tgt_texts[:n]]).score
    print(f'Score Bleu de la traduction du corpus Test par la méthode Transformer = {translation_quality_Transformer:.1f}%')
    tps3 = time.time()
    print(" -> durée de traduction et de calcul du score :",int(tps3-tps2)," secondes\n")


Score Bleu de la traduction du corpus Test par la méthode RNN = 94.4%
 -> durée de traduction et de calcul du score : 291  secondes

Score Bleu de la traduction du corpus Test par la méthode Transformer = 94.1%
 -> durée de traduction et de calcul du score : 135  secondes

